In [67]:
'''
Select an individual feature by attribute value from one feature class, then select features from another feature class by distance from the original feature. 
Originally used for selecting census block groups within a specified distance of a point feature. 
'''
def select_feature_by_attribute(focus_features, focus_field, focus_value, target_features, selection_distance, output_path, overwrite): 
    
    # Construct SQL 
    sql = f"{focus_field} = '{focus_value}'"

    # Apply attribute selection 
    focus_selection, count          = arcpy.management.SelectLayerByAttribute(focus_features, 'NEW_SELECTION', sql)
    
    target_selection, merge, count  = arcpy.management.SelectLayerByLocation(target_features, 'WITHIN_A_DISTANCE', focus_features, selection_distance, 'NEW_SELECTION')
    
    # Export target selection 
    if arcpy.Exists(output_path) and overwrite == True: 
        arcpy.conversion.ExportFeatures(target_selection, output_path)
    else: 
        print(f'Feature class exists: {output_path}')
    
    return output_path

In [ ]:
'''
Parameters
'''

import os 

# Focus inputs 
focus_features     = r'C:\DALE\Personal\Minnesota\MyProject\ACS.gdb\Focus_Points'
focus_field        = 
focus_value        =

# Target inputs 
target_features    =
selection_distance =

# Output 
output_path        =
overwrite          =

In [ ]:
'''
Run 
'''
output_path = select_feature_by_attribute(focus_features, focus_field, focus_value, target_features, selection_distance, output_path, overwrite)